In [14]:
import csv
from datetime import datetime
from enum import Enum
import os
from pathlib import Path
from typing import Union, List, Iterable

from matplotlib import pyplot as plt
import pandas as pd
import requests
import seaborn as sns

### Constants

In [14]:
HOME_DIR = Path().cwd()
REPO_PATH = HOME_DIR / "repos"
REPO_PATH.mkdir(exist_ok=True)
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%SZ'
PLOT_IMAGES_PATH = HOME_DIR / "plot_images"
PLOT_IMAGES_PATH.mkdir(exist_ok=True)

access_token = None

if access_token:
    AUTHORIZATION_HEADER = {"Authorization": f"Bearer {access_token}",
                            "Accept": "application/vnd.github.v3+json",
                            "X-Github-Api-Version": "2022-11-28"}
else:
    AUTHORIZATION_HEADER = {"Accept": "application/vnd.github.v3+json",
                            "X-Github-Api-Version": "2022-11-28"}


<h5>Enum classes</h5>

In [15]:
class MenuOptions(Enum):
    GET_REPO = 1
    SHOW_REPOS = 2
    VIZ_REP_REPOS = 3
    USERS_CORRELATION = 4
    EXIT = 5


class RepoMenuOptions(Enum):
    SHOW_PULL_REQUESTS = 1
    SHOW_SUMMARY = 2
    VIZ_REP = 3
    CORRELATION = 4
    BACK_TO_MAIN_MENU = 5


### Helper functions

In [16]:
def getPullRequests(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
    print(url)
    response = requests.get(url, headers=AUTHORIZATION_HEADER)

    if response.status_code != 200:
        print(f"Status code: {response.status_code}\n"
              f"{response.text}")
        return None
    else:
        resp_list = response.json()
    ret = []
    for item in resp_list:
        pr = PullRequest.parse(item)
        print("  Collected PR: ", pr.title)
        ret.append(pr)
    return ret


def getPullRequestInfo(owner, repo, number):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{number}"
    response = requests.get(url, headers=AUTHORIZATION_HEADER)

    if response.status_code != 200:
        print(f"Failed to retrieve pull requests. Status code: {response.status_code}\n"
              f"{response.text}")
        return None
    else:
        resp_dict = response.json()
        pr_info = PRStatus.parse(resp_dict)
        return pr_info

def get_repository(owner, repo) -> Union[None, "GitHubRepository"]:
    url = f"https://api.github.com/repos/{owner}/{repo}"
    response = requests.get(url, headers=AUTHORIZATION_HEADER)

    if response.status_code != 200:
        print(f"Status code: {response.status_code}\n"
              f"{response.text}")
        return None
    resp_dict = response.json()
    return GitHubRepository.parse(resp_dict)

def getUserInfo(user):
    url = f"https://api.github.com/users/{user}"

    user_info = None
    response = requests.get(url, headers=AUTHORIZATION_HEADER)

    if response.status_code != 200:
        print(f"Status code for {url}: {response.status_code}\n"
              f"{response.text}")
        return None
    else:
        resp_dict = response.json()
        user_info = UserInfo.parse(resp_dict)
        # contributions
        url = f"https://api.github.com/users/{user}/events"
        response = requests.get(url, headers=AUTHORIZATION_HEADER)

    events = []
    created_year = 0
    curr_year = 0
    contributions = 0
    pull_requests = 0

    if response.status_code != 200:
        print(f"Status code for {url}: {response.status_code}")
    else:
        events = response.json()
        contributions = 0

    for event in events:
        if event.get("type", "null") == "PullRequestEvent":  # check if the event is a pull request
            # check if the contribution was made this year
            created_at = event.get("created_at", "")

            if created_at != "":
                created_year = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ").year
                curr_year = datetime.now().year

            if curr_year == created_year:
                pull_requests += 1

        if event.get("type", "null") == "PushEvent":  # check if the event is a push
            # print(json.dumps(event, indent=4))

            # check if the contribution was made this year
            created_at = event.get("created_at", "")

            if created_at != "":
                created_year = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ").year
                curr_year = datetime.now().year

            if curr_year == created_year:
                contributions += 1

    if user_info is not None:
        user_info.set_contributions(contributions)
        user_info.set_pull_requests(pull_requests)

    return user_info

def save_as_csv(file_path, data: List[str], mode="a", header=""):
    write_header = False
    if not os.path.exists(file_path):
        write_header = True

    with open(file_path, mode=mode) as file:
        csv_writer = csv.writer(file)
        if write_header:
            csv_writer.writerow(header)
        csv_writer.writerow(data)


<h5>User input Handler</h5>

In [17]:

def accept_repo_input() -> tuple[str, str]:
    """
    Function to accept user input for owner name and repo name.
    """
    owner_name = input("Enter owner name: ")
    repo_name = input("Enter repo name: ")
    return owner_name, repo_name

class UserInputHandler:
    """
    Class to handle user inputs.
    """

    @staticmethod
    def get_repo() -> Union[MenuOptions, RepoMenuOptions, tuple[str, str], int]:
        """
        Function to accept user input and call the specific data acceptor function based on the menu choice.
        """

        return accept_repo_input()

    @staticmethod
    def select_repo(valid_number_range: Iterable) -> int:
        """
        Function to accept user input for owner name and repo name.
        """
        user_input = input("Select a repository from the list above:")
        if user_input not in valid_number_range:
            print("You have entered an invalid choice for the current menu.\n" "Please try again.\n")
            return UserInputHandler.select_repo(valid_number_range)
        return int(user_input)


### Classes

<h5>PULL REQUESTS</h5>

In [18]:
class PullRequest:
    def __init__(self, user, title, number, body, state, created_at, closed_at):
        self.user = user
        self.title = title
        self.number = number
        self.body = body
        self.state = state
        self.created_at = created_at
        self.closed_at = closed_at
        self.pr_status: Union[None, PRStatus] = None

    def store_status(self, owner_name, repo_name):
        self.pr_status = getPullRequestInfo(owner_name, repo_name, self.number)

    def save_as_csv(self, repository):
        header = ['number', 'repo_name', 'repo_owner', 'title', 'state', 'created_at', 'closed_at', 'user', 'commits',
                  'additions', 'deletions', 'changed_files']
        data = [
            self.number, repository.name, repository.owner, self.title, self.state, self.created_at,
            self.closed_at, self.user, self.pr_status.commits, self.pr_status.additions, self.pr_status.deletions,
            self.pr_status.changed_files]
        save_as_csv(REPO_PATH / f"{repository.owner}-{repository.name}.csv", data, header=header)

    @staticmethod
    def parse(response_dict):
        title = response_dict["title"] if "title" in response_dict else None
        user = response_dict["user"]["login"] if "user" in response_dict and "login" in response_dict["user"] else None
        number = response_dict["number"] if "number" in response_dict else None
        body = response_dict["body"] if "body" in response_dict else None
        state = response_dict["state"] if "state" in response_dict else None
        created_at = response_dict["created_at"] if "created_at" in response_dict else None
        closed_at = response_dict["closed_at"] if "closed_at" in response_dict else None

        return PullRequest(user, title, number, body, state, created_at, closed_at)

    def __str__(self):
        return f"{self.user}/{self.title}: ({self.state}) ({self.number}) ({self.created_at}) ({self.closed_at})"


class PRStatus:
    def __init__(self, number, commits, additions, deletions, changed_files, author_association):
        self.number = number
        self.commits = commits
        self.additions = additions
        self.deletions = deletions
        self.changed_files = changed_files
        self.author_association = author_association

    @staticmethod
    def parse(response_dict):
        number = response_dict["number"] if "number" in response_dict else None
        commits = response_dict["commits"] if "commits" in response_dict else None
        additions = response_dict["additions"] if "additions" in response_dict else None
        deletions = response_dict["deletions"] if "deletions" in response_dict else None
        changed_files = response_dict["changed_files"] if "changed_files" in response_dict else None
        author_association = response_dict.get("author_association", None)

        return PRStatus(number, commits, additions, deletions, changed_files, author_association)

    def __str__(self):
        return f"PullRequest #{self.number}: commits = ({self.commits}) additions = ({self.additions}) deletions = ({self.deletions}) file changes = ({self.changed_files})"


<h5>REPOS</h5>

In [19]:
class GitHubRepository:
    def __init__(self, name, owner, description, homepage, license, forks, watchers, date_of_collection):
        self.name = name
        self.owner = owner
        self.description = description
        self.homepage = homepage
        self.license = license  # License could be another class with its own attributes
        self.forks = forks
        self.watchers = watchers
        self.date_of_collection = date_of_collection

        self.pull_requests: List[PullRequest] = []  # List of PullRequest objects
        self.users: List[UserInfo] = []

    def store_pull_requests(self):
        self.pull_requests = getPullRequests(self.owner, self.name) or []
        for pull_request in self.pull_requests:
            pull_request.store_status(self.owner, self.name)
            user = getUserInfo(pull_request.user)
            if user is not None:
                self.users.append(user)

    @staticmethod
    def parse(response_dict):
        name = response_dict["name"] if "name" in response_dict else None
        owner = (
            response_dict["owner"]["login"] if "owner" in response_dict and "login" in response_dict["owner"] else None
        )
        description = response_dict["description"] if "description" in response_dict else None
        homepage = response_dict["homepage"] if "homepage" in response_dict else None
        forks = response_dict["forks"] if "forks" in response_dict else None
        watchers = response_dict["watchers"] if "watchers" in response_dict else None
        license = (
            response_dict["license"]["name"]
            if "license" in response_dict
               and response_dict["license"] is not None
               and "name" in response_dict["license"]
            else None
        )

        time = datetime.now()
        formatted_time = time.strftime("%Y-%m-%dT%H:%M:%SZ")

        return GitHubRepository(name, owner, description, homepage, license, forks, watchers, formatted_time)

    def show_summary(self):
        """
        Function to show summary of the repository.
        Summary must include the following things:
            - Number of pull requests in `open` state
            - Number of pull requests in `closed` state
            - Number of users
            - Date of the oldest pull request
        """
        open_prs = 0
        closed_prs = 0
        oldest_pr_date = datetime.now()

        for pull_request in self.pull_requests:
            if pull_request.state == "open":
                open_prs += 1
            elif pull_request.state == "closed":
                closed_prs += 1
            if datetime.strptime(pull_request.created_at, DATETIME_FORMAT) < oldest_pr_date:
                oldest_pr_date = datetime.strptime(pull_request.created_at, DATETIME_FORMAT)

        print(f"Number of pull requests in `open` state: {open_prs}")
        print(f"Number of pull requests in `closed` state: {closed_prs}")
        print(f"Number of users: {len(self.users)}")
        print(f"Date of the oldest pull request: {oldest_pr_date}")

    def __str__(self):
        return f"{self.owner}/{self.name}: license = [{self.license}] homepage = [{self.homepage}] forks = ({self.forks}) watchers = ({self.watchers}) date of collection: {self.date_of_collection}"

    def save_as_csv(self):
        repo_header = ['name', 'owner', 'description', 'homepage', 'license', 'forks', 'watchers', 'date_of_collection']
        data = [
            self.name, self.owner, self.description, self.homepage, self.license, self.forks, self.watchers,
            self.date_of_collection
        ]
        save_as_csv(REPO_PATH / "repositories.csv", data, header=repo_header)
        for pull_request in self.pull_requests:
            pull_request.save_as_csv(self)
        for user in self.users:
            user.save_as_csv()


<h5>USERS</h5>

In [20]:
class UserInfo:
    def __init__(self, name, no_of_repos, followers, following):
        self.name = name
        self.no_of_repos = no_of_repos
        self.followers = followers
        self.following = following

        self.contributions = 0
        self.pull_requests = 0

    @staticmethod
    def parse(response_dict):
        name = response_dict["name"] if "name" in response_dict else None
        no_of_repos = response_dict["public_repos"] if "public_repos" in response_dict else None
        followers = response_dict["followers"] if "followers" in response_dict else None
        following = response_dict["following"] if "following" in response_dict else None

        return UserInfo(name, no_of_repos, followers, following)

    def set_contributions(self, contributions):
        self.contributions = contributions

    def set_pull_requests(self, pull_requests):
        self.pull_requests = pull_requests

    def __str__(self):
        return f"{self.name}: repos = {self.no_of_repos}, followers = {self.followers}, following = {self.following}, contributions in last year = {self.contributions}"

    def save_as_csv(self):
        header = ['name', 'no_of_repos', 'followers', 'following', 'contributions']
        data = [self.name, self.no_of_repos, self.followers, self.following, self.contributions]
        save_as_csv(REPO_PATH / "users.csv", data, header=header)


<h3>Vizualization - Functions</h3>

<h5>PULL REQUEST VISUALIZATION</h5>

In [21]:
def create_df_PR_numerical_data(repo: GitHubRepository):
    """
    Function to create a dataframe of all the numerical data from the pull requests of a repository.
    """
    df = pd.DataFrame(
        {
            "Commits": [pr.pr_status.commits for pr in repo.pull_requests],
            "Additions": [pr.pr_status.additions for pr in repo.pull_requests],
            "Deletions": [pr.pr_status.deletions for pr in repo.pull_requests],
            "Changed Files": [pr.pr_status.changed_files for pr in repo.pull_requests],
        }
    )
    return df


def generate_correlation_plot_PR_numerical_data(repo: GitHubRepository):
    """
    Function to generate and save a correlation plot
    for all the numerical data from the pull requests of a repository.
    """
    df = create_df_PR_numerical_data(repo)
    plt.figure(figsize=(10, 10))
    corr = df.corr()
    sns.heatmap(corr, annot=True, cmap="coolwarm")
    plt.title(f"PR Correlation plot for {repo.owner}/{repo.name}")
    plt.savefig(PLOT_IMAGES_PATH / f"{repo.owner}-{repo.name}-correlation.png")
    plt.show()

<h5>REPO VISUALIZATION</h5>

In [22]:
def create_commits_df(repo: GitHubRepository):
    """
    Create a dataframe with 2 columns:
        1. Closed_PR
        2. Open_PR
    Fill the cells with the number of commits for each PR.
    """
    commits_df = pd.DataFrame(columns=["Closed_PR", "Open_PR"])
    for pull_request in repo.pull_requests:
        if pull_request.state == "closed":
            commits_df = pd.concat(
                [
                    commits_df,
                    pd.DataFrame(
                        {
                            "Closed_PR": [pull_request.pr_status.commits],
                            "Open_PR": [0],
                        }
                    ),
                ]
            )
        else:
            commits_df = pd.concat(
                [
                    commits_df,
                    pd.DataFrame(
                        {
                            "Closed_PR": [0],
                            "Open_PR": [pull_request.pr_status.commits],
                        }
                    ),
                ]
            )
    return commits_df.astype(int)


def create_changes_df(repo: GitHubRepository):
    """
    Create a dataframe with 2 columns:
        1. Closed_PR
        2. Open_PR
    Fill the cells with the number of changes for each PR.
    """
    changes_df = pd.DataFrame(columns=["Closed_PR", "Open_PR"])
    for pull_request in repo.pull_requests:
        if pull_request.state == "closed":
            changes_df = pd.concat(
                [
                    changes_df,
                    pd.DataFrame(
                        {
                            "Closed_PR": [pull_request.pr_status.additions + pull_request.pr_status.deletions],
                            "Open_PR": [0],
                        }
                    ),
                ]
            )
        else:
            changes_df = pd.concat(
                [
                    changes_df,
                    pd.DataFrame(
                        {
                            "Closed_PR": [0],
                            "Open_PR": [pull_request.pr_status.additions + pull_request.pr_status.deletions],
                        }
                    ),
                ]
            )
    return changes_df.astype(int)


def create_author_association_vs_changed_files_df(repo: GitHubRepository):
    """
    Create a dataframe with 2 columns:
        1. Author Association
        2. Changed Files
    Fill the cells with the number of changes for each PR.
    """
    changes_df = pd.DataFrame(columns=["Author Association", "Changed Files"])
    for pull_request in repo.pull_requests:
        changes_df = pd.concat(
            [
                changes_df,
                pd.DataFrame(
                    {
                        "Author Association": [pull_request.pr_status.author_association],
                        "Changed Files": [pull_request.pr_status.changed_files],
                    }
                ),
            ]
        )
    # group by Author Association
    changes_df = changes_df.groupby("Author Association").sum().reset_index()
    changes_df['Changed Files'] = changes_df['Changed Files'].astype(int)
    return changes_df


def create_df_additions_deletions(repo: GitHubRepository):
    """
    Create a dataframe to show the relationship between the number of additions and deletions for each PR.
    """
    changes_df = pd.DataFrame(columns=["Additions", "Deletions"])
    for pull_request in repo.pull_requests:
        changes_df = pd.concat(
            [
                changes_df,
                pd.DataFrame(
                    {
                        "Additions": [pull_request.pr_status.additions],
                        "Deletions": [pull_request.pr_status.deletions],
                    }
                ),
            ]
        )
    return changes_df.astype(int)


def generate_comparison_plots(repo_list: List[GitHubRepository]):
    """
    Generate and save the following comparison plots:
        1. n total PR per day
        2. n open vs closed PR per day
        3. n users per repo
    """
    # construct the dataframe for all repositories
    pr_df = pd.DataFrame()
    for repo in repo_list:
        for pull_request in repo.pull_requests:
            pr_df = pd.concat(
                [
                    pr_df,
                    pd.DataFrame(
                        {
                            "Repo": [repo.name],
                            "User": [pull_request.user],
                            "State": [pull_request.state],
                            "Create_Date": [pull_request.created_at],
                            "Closed_Date": [pull_request.closed_at],
                        }
                    ),
                ]
            )

    if pr_df.empty:
        print("No pull requests found.")
        return
    # convert the date columns to datetime
    pr_df["Create_Date"] = pd.to_datetime(pr_df["Create_Date"])
    pr_df["Closed_Date"] = pd.to_datetime(pr_df["Closed_Date"])

    plt.figure(figsize=(10, 10))
    # line graph for n total PR per day
    pr_df.groupby("Create_Date").size().plot()
    plt.xlabel("Date")
    plt.ylabel("Number of PRs")
    plt.title("Total PR per day")
    plt.savefig(PLOT_IMAGES_PATH / "total_PR_per_day.png")
    plt.show()

    plt.figure(figsize=(10, 10))
    # line graph for n open vs closed PR per day
    pr_df.groupby(["Create_Date", "State"]).size().unstack().plot()
    plt.xlabel("Date")
    plt.ylabel("Number of PRs")
    plt.title("Open vs Closed PR per day")
    plt.savefig(PLOT_IMAGES_PATH / "open_vs_closed_PR_per_day.png")
    plt.show()

    plt.figure(figsize=(10, 10))
    # bar graph for n users per repo
    pr_df.groupby("Repo")["User"].nunique().plot(kind="bar")
    plt.xlabel("Repository")
    plt.ylabel("Number of Users")
    plt.title("Users per repo")
    plt.savefig(PLOT_IMAGES_PATH / "users_per_repo.png")
    plt.show()


def generate_PR_vs_Commits_boxplot(repo: GitHubRepository):
    """
    Create a boxplot to show the number of commits for each PR.
    Also save the boxplot as an image.
    """
    plt.figure(figsize=(10, 10))
    commits_df = create_commits_df(repo)
    commits_df.boxplot()
    plt.xlabel("PR State")
    plt.ylabel("Commits")
    plt.title("PR State vs Commits")
    plt.savefig(PLOT_IMAGES_PATH / f"{repo.owner}-{repo.name}-PR_vs_Commits.png")
    plt.show()


def generate_PR_vs_Changes_boxplot(repo: GitHubRepository):
    """
    Create a boxplot to show the number of changes for each PR.
    Also save the boxplot as an image.
    """
    plt.figure(figsize=(10, 10))
    changes_df = create_changes_df(repo)
    changes_df.boxplot()
    plt.xlabel("PR State")
    plt.ylabel("Changes")
    plt.title("PR State vs Changes")
    plt.savefig(PLOT_IMAGES_PATH / f"{repo.owner}-{repo.name}-PR_vs_Changes.png")
    plt.show()


def generate_Author_Association_vs_Changed_Files_boxplot(repo: GitHubRepository):
    """
    Create a boxplot to show the number of changes for each PR.
    Also save the boxplot as an image.
    """
    plt.figure(figsize=(10, 10))
    changes_df = create_author_association_vs_changed_files_df(repo)
    changes_df.boxplot()
    plt.title("Author Association vs Changed Files")
    plt.savefig(PLOT_IMAGES_PATH / f"{repo.owner}-{repo.name}-Author_Association_vs_Changed_Files.png")
    plt.show()


def generate_Additions_vs_Deletions_scatterplot(repo: GitHubRepository):
    """
    Create a scatterplot to show the relationship between the number of additions and deletions for each PR.
    Also save the scatterplot as an image.
    """
    plt.figure(figsize=(10, 10))
    changes_df = create_df_additions_deletions(repo)

    plt.scatter(changes_df["Additions"], changes_df["Deletions"])
    plt.xlabel("Additions")
    plt.ylabel("Deletions")
    plt.title("Additions vs Deletions")
    plt.savefig(PLOT_IMAGES_PATH / f"{repo.owner}-{repo.name}-Additions_vs_Deletions.png")
    plt.show()


def use_all_plot_generators_for_repo(repo: GitHubRepository):
    """
    Function to generate all the plots for a repository.
    """
    generate_PR_vs_Commits_boxplot(repo)
    generate_PR_vs_Changes_boxplot(repo)
    generate_Author_Association_vs_Changed_Files_boxplot(repo)
    generate_Additions_vs_Deletions_scatterplot(repo)


<h5>USERS VISUALIZATION</h5>

In [23]:
def create_df_users_numerical_data(repo: GitHubRepository):
    """
    Function to create a dataframe with the following columns:
        1. Followers
        2. Following
        3. Number of PRs
        4. number of contributions
    """
    df = pd.DataFrame(
        {
            "Followers": [user.followers for user in repo.users],
            "Following": [user.following for user in repo.users],
            "Number of PRs": [user.pull_requests for user in repo.users],
            "Number of Contributions": [user.contributions for user in repo.users],
        }
    )
    return df


def generate_users_correlation_plot(repo_list: List[GitHubRepository]):
    """
    Function to generate and save a correlation plot
    for all the numerical data from the pull requests of a repository.
    """
    df = pd.DataFrame()
    for repo in repo_list:
        df = pd.concat([df, create_df_users_numerical_data(repo)], ignore_index=True)

    plt.figure(figsize=(10, 10))
    corr_matrix = df.corr()
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
    plt.title(f"Users Correlation plot")
    plt.savefig(PLOT_IMAGES_PATH / "users-correlation.png")
    plt.show()


<h3><b>PROGRAM LOOP</b></h3>

In [24]:

class Application:
    def __init__(self):
        self.repos = []

    @staticmethod
    def display_menu(display_option: str):
        main_menu = (
            "1. Get repository",
            "2. Show Collected Repositories",
            "3. Create & Store the visual representation of all the repositories",
            "4. Calculate correlation among the stored users data",
            "5. Exit",
        )

        repo_submenu = (
            "1. Show all pull requests",
            "2. Show summary of repository",
            "3. Create & Store the visual representation of the repository data",
            "4. Calculate correlation between all numeric data in the repository",
            "5. Return to main menu",
        )
        if display_option == "main":
            for option in main_menu:
                print(option)

        elif display_option == "repo":
            for option in repo_submenu:
                print(option)

        else:
            raise ValueError("Invalid menu display option")

    @staticmethod
    def accept_user_input():
        user_input = input("Enter your choice: ")
        return user_input

    @staticmethod
    def validate_user_input_choice(menu_choice: str, user_choice: str):
        """
        Function to verify if the entered userr choice is valid on given the current menu displayed.
        """
        valid_choices_mapping = {"main": map(str, range(1, 6)), "repo": map(str, range(1, 6))}
        valid_choices = valid_choices_mapping.get(menu_choice, ())
        if user_choice not in valid_choices:
            return False
        return True

    def display_and_accept_menu_choice(self, menu_choice: str) -> Union[MenuOptions, RepoMenuOptions]:
        """
        Function to handle the following tasks:
            1. Display the menu/sub menu
            2. Ask for user input
            3. Validate user input
        """
        self.display_menu(menu_choice)
        user_input = self.accept_user_input()
        is_valid = self.validate_user_input_choice(menu_choice, user_input)
        if not is_valid:
            print("You have entered an invalid choice for the current menu.\n" "Please try again.\n")
            return self.display_and_accept_menu_choice(menu_choice)

        if menu_choice == "main":
            print("\n")
            return MenuOptions(int(user_input))
        else:
            print("\n")
            return RepoMenuOptions(int(user_input))

    def run_app(self):
        """
        Function to run the application loop and keep showing the menu/submenu
        options until user choose to exit the application.
        """
        current_menu_choice = "main"
        selected_repo_index = None
        while 1:
            user_input = self.display_and_accept_menu_choice(current_menu_choice)
            if current_menu_choice == "main":
                if user_input == MenuOptions.GET_REPO:  # 1
                    owner_name, repo_name = UserInputHandler.get_repo()
                    github_repo = get_repository(owner_name, repo_name)

                    if github_repo is not None:
                        github_repo.store_pull_requests()
                        self.repos.append(github_repo)
                        github_repo.save_as_csv()
                        print(f"Repository {github_repo.name} added successfully")
                    else:
                        print(f"Failed to retrieve repository {repo_name} from {owner_name}")

                elif user_input == MenuOptions.SHOW_REPOS:  # 2

                    if self.repos:
                        for index, repo in enumerate(self.repos, 1):
                            print(f"{index}. {repo}")
                        user_input = UserInputHandler.select_repo(tuple(map(str, range(1, len(self.repos) + 1))))
                        current_menu_choice = "repo"
                        selected_repo_index = user_input - 1
                    else:
                        print("No repositories found. Please add a repository first.")
                        current_menu_choice = "main"
                        selected_repo_index = None

                elif user_input == MenuOptions.VIZ_REP_REPOS:  # 3
                    if not self.repos:
                        print("Please add atleast 1 repository to use this option.")
                        continue

                    try:
                        generate_comparison_plots(self.repos)
                    except Exception as err:
                        print("Failed to create repository correlation plots.", err)

                elif user_input == MenuOptions.USERS_CORRELATION:  # 4
                    if not self.repos:
                        print("Please add at-least 1 repository to use this option.")
                        continue

                    try:
                        generate_users_correlation_plot(self.repos)
                    except Exception as err:
                        print("Failed to create users correlation plot.", err)

                elif user_input == MenuOptions.EXIT:  # 5
                    break

            elif current_menu_choice == "repo":
                if user_input == RepoMenuOptions.SHOW_PULL_REQUESTS:  # 1
                    # show all pull-requests from a certain repository
                    repo = self.repos[selected_repo_index]
                    for pr in repo.pull_requests:
                        print(pr)
                elif user_input == RepoMenuOptions.SHOW_SUMMARY:  # 2
                    try:
                        repo = self.repos[selected_repo_index]
                        repo.show_summary()
                    except Exception as err:
                        print("Failed to show repository summary, due the error:", err)

                elif user_input == RepoMenuOptions.VIZ_REP:  # 3
                    try:
                        repo = self.repos[selected_repo_index]
                        use_all_plot_generators_for_repo(repo)
                    except Exception as err:
                        print("Failed to generate vizual representation of the repository, due the error:", err)

                elif user_input == RepoMenuOptions.CORRELATION:  # 4
                    try:
                        repo = self.repos[selected_repo_index]
                        generate_correlation_plot_PR_numerical_data(repo)
                    except Exception as err:
                        print("Failed to generate correlation plot for repository, due the error:", err)

                elif user_input == RepoMenuOptions.BACK_TO_MAIN_MENU:  # 5
                    current_menu_choice = "main"
                    selected_repo_index = None
            print("-" * 35)


<h5>Start the program loop</h5>

In [25]:
try:
    app = Application()
    app.run_app()
except KeyboardInterrupt:
    print("Program closed!")
except Exception as e:
    print(f"Program crashed due to: {e}")

1. Get repository
2. Show Collected Repositories
3. Create & Store the visual representation of all the repositories
4. Calculate correlation among the stored users data
5. Exit
You have entered an invalid choice for the current menu.
Please try again.

1. Get repository
2. Show Collected Repositories
3. Create & Store the visual representation of all the repositories
4. Calculate correlation among the stored users data
5. Exit
You have entered an invalid choice for the current menu.
Please try again.

1. Get repository
2. Show Collected Repositories
3. Create & Store the visual representation of all the repositories
4. Calculate correlation among the stored users data
5. Exit
You have entered an invalid choice for the current menu.
Please try again.

1. Get repository
2. Show Collected Repositories
3. Create & Store the visual representation of all the repositories
4. Calculate correlation among the stored users data
5. Exit
You have entered an invalid choice for the current menu.
Ple